In [ ]:
from google.colab import drive, files
from pathlib import Path
import sys
drive.mount('/content/drive', force_remount = True)
DATADIR = Path("/content/drive/MyDrive/Colab Notebooks/42-678_Early_Onset_Diabetes_Prediction/diabetes.csv")
sys.path.append(str(DATADIR.parent))

Mounted at /content/drive


In [ ]:
# Importing essential libraries
import numpy as np
import pandas as pd
import pickle

In [ ]:
# Loading the dataset
df = pd.read_csv(DATADIR)

In [ ]:
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [ ]:
# Renaming DiabetesPedigreeFunction as DPF
df = df.rename(columns={'DiabetesPedigreeFunction':'DPF'})

In [ ]:
# Replacing the 0 values from ['Glucose','BloodPressure','SkinThickness','Insulin','BMI'] by NaN
df_copy = df.copy(deep=True)
df_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)


In [ ]:
# Replacing NaN value by mean, median depending upon distribution
df_copy['Glucose'].fillna(df_copy['Glucose'].mean(), inplace=True)
df_copy['BloodPressure'].fillna(df_copy['BloodPressure'].mean(), inplace=True)
df_copy['SkinThickness'].fillna(df_copy['SkinThickness'].median(), inplace=True)
df_copy['Insulin'].fillna(df_copy['Insulin'].median(), inplace=True)
df_copy['BMI'].fillna(df_copy['BMI'].median(), inplace=True)

In [ ]:
# Model Building
from sklearn.model_selection import train_test_split
X = df.drop(columns='Outcome')
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [ ]:
X_test

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DPF,Age
661,1,199,76,43,0,42.9,1.394,22
122,2,107,74,30,100,33.6,0.404,23
113,4,76,62,0,0,34.0,0.391,25
14,5,166,72,19,175,25.8,0.587,51
529,0,111,65,0,0,24.6,0.660,31
...,...,...,...,...,...,...,...,...
253,0,86,68,32,0,35.8,0.238,25
622,6,183,94,0,0,40.8,1.461,45
235,4,171,72,0,0,43.6,0.479,26
351,4,137,84,0,0,31.2,0.252,30


In [ ]:
# Creating Random Forest Model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=650)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=650)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
SVC_model = SVC()
KNN_model = KNeighborsClassifier(n_neighbors=25)

In [ ]:
SVC_model.fit(X_train, y_train)
KNN_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=25)

In [ ]:
y_pred = SVC_model.predict(X_test)

In [ ]:
y_pred = KNN_model.predict(X_test)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[46  5]
 [ 8 18]]
              precision    recall  f1-score   support

           0       0.85      0.90      0.88        51
           1       0.78      0.69      0.73        26

    accuracy                           0.83        77
   macro avg       0.82      0.80      0.81        77
weighted avg       0.83      0.83      0.83        77

0.8311688311688312


In [ ]:
# Creating a pickle file for the classifier
filename = '/content/drive/MyDrive/Colab Notebooks/42-678_Early_Onset_Diabetes_Prediction/diabetes-prediction-rfc-model.pkl'
pickle.dump(SVC_model, open(filename, 'wb'))